### SETUP INICIAL DO PROJETO

In [1]:

#importação das bibliotecase e pacotes necessários para a análise

import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pandas_gbq as gbq
import re
import seaborn as sns
from dotenv import load_dotenv
from google.cloud import bigquery
from google.cloud.bigquery_storage import BigQueryReadClient
from google.oauth2 import service_account


# carregar o .env com as credenciais
load_dotenv("/mnt/c/Users/wrpen/OneDrive/Desktop/df_lh/.env")


# detectar ambiente: como eu estou usando wsl-ubuntu, no VS Code  -  Windows, estava dando conflitos de path
if os.name == "nt":  # se Windows
    credentials_path = r"C:\Temp\desafiolh-445818-3cb0f62cb9ef.json"
else:  # se WSL/Linux
    credentials_path = "/mnt/c/Temp/desafiolh-445818-3cb0f62cb9ef.json"


# parâmetros injetados pelo Papermill ou definidos manualmente, caso não existam no ambiente
if 'tables_to_process' not in locals():
    tables_to_process = [
        "desafioadventureworks-446600.raw_data.production-productinventory"       
    ]

if 'output_dataset' not in locals():
    output_dataset = "desafioadventureworks-446600.raw_data_cleaned"


# configs do cliente BigQuery: input de project e location de acordo com dados no Bigquery
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=os.getenv("BIGQUERY_PROJECT"), location="us-central1")


#doc: tables_to_process: lista de tabelas que serão processadas
#     output_dataset: nome do dataset onde os dados processados serão armazenados, neste caso, raw_data_cleaned

In [2]:
# Print com a tabela que vai ser processada nesse notebook

print("Tabelas a processar:", tables_to_process)

Tabelas a processar: ['desafioadventureworks-446600.raw_data.production-productinventory']


In [3]:
# Nome do dataset no Bigquery com os dados brutos (.csv) extraídos pelo Meltano 
dataset_id = 'raw_data'
print(dataset_id)

# Lista de tabelas do dataset raw_data no Bigquery
tables = client.list_tables('raw_data')
print("Tabelas disponíveis:")
for table in tables:
    print(table.table_id)

raw_data
Tabelas disponíveis:
humanresources-employee
person-address
person-businessentity
person-person
person-stateprovince
production-location
production-product
production-productcategory
production-productcosthistory
production-productinventory
production-productsubcategory
purchasing-purchaseorderdetail
purchasing-purchaseorderheader
purchasing-vendor
sales-creditcard
sales-customer
sales-salesorderdetail
sales-salesorderheader
sales-salesperson
sales-salesterritory
sales-store


# Exploratory Data Analysis (EDA) e Data Cleaning

### Glossário dos dados:

O termo ''doc:'', situado no rodapé de algumas cells, indica algo como:

- documentação: documentar decisões, análises e resultados;

- abreviações de termos, como bkp, df, entre outros.

In [4]:
# Setup inicial do df para realizar a EDA 

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 10000)
pd.options.display.float_format = '{:.2f}'.format


#doc: df = dataframe  

In [5]:
# Dicionário para armazenar os df processados
df_processados = {}

# Iteração das tabelas e armazenamento em df
for input_table in tables_to_process:
    print(f"Processando tabela: {input_table}")
    
    table_name = input_table.split(".")[-1].replace("-", "_")  
    
    print("Lendo os dados do BigQuery...")
    query = f"SELECT * FROM `{input_table}`"
    table_data = client.query(query).to_dataframe()
    
    df_processados[table_name] = table_data
    print(f"Tabela {table_name} processada e armazenada com sucesso.")


print("Todas as tabelas foram processadas com sucesso!")

Processando tabela: desafioadventureworks-446600.raw_data.production-productinventory
Lendo os dados do BigQuery...
Tabela production_productinventory processada e armazenada com sucesso.
Todas as tabelas foram processadas com sucesso!


In [6]:
# Listar todas as variáveis criadas dinamicamente
for table_name in df_processados.keys():
    print(f"Variável criada: {table_name}")  

Variável criada: production_productinventory


In [7]:
# Atribuir o df a uma variável com nome mais simples
production_productinventory = df_processados['production_productinventory']

print(f"Colunas: {production_productinventory.shape[1]}\nLinhas: {production_productinventory.shape[0]}")

Colunas: 7
Linhas: 13897


In [8]:
# Identificar duplicadas
duplicadas = production_productinventory[production_productinventory.duplicated(subset=['productid'], keep=False)]

# Verificar se existem duplicadas
if not duplicadas.empty:
    
    duplicadas_ordenadas = duplicadas.sort_values(by=['productid', 'modifieddate'])

    print("duplicadas ordenadas:")
    print(duplicadas_ordenadas)
else:
    print("Não foram encontradas duplicadas.")

duplicadas ordenadas:
       productid  locationid shelf  bin  quantity                               rowguid              modifieddate
240            1           1     A    1       408  47a24246-6c43-48eb-968f-025738a8a410 2014-08-08 00:00:00+00:00
241            1           1     A    1       408  47a24246-6c43-48eb-968f-025738a8a410 2014-08-08 00:00:00+00:00
242            1           1     A    1       408  47a24246-6c43-48eb-968f-025738a8a410 2014-08-08 00:00:00+00:00
243            1           1     A    1       408  47a24246-6c43-48eb-968f-025738a8a410 2014-08-08 00:00:00+00:00
244            1           1     A    1       408  47a24246-6c43-48eb-968f-025738a8a410 2014-08-08 00:00:00+00:00
...          ...         ...   ...  ...       ...                                   ...                       ...
11745        999          60   N/A    0       116  aff43c54-af78-4635-8f8a-733a1fc2d085 2013-04-30 00:00:00+00:00
11746        999          60   N/A    0       116  aff43c54-af78-4

In [9]:
# Remover duplicadas* 
production_productinventory = production_productinventory.drop_duplicates(subset=['productid', 'locationid'], keep='first')

print(f"Linhas após remover duplicatas com base em 'productid' e 'locationid': {len(production_productinventory)}")

#bkp dos dados brutos
raw_data_bkp_sem_duplicadas = production_productinventory.copy()


#doc: bkp = backup (cópia)
#doc*: mantendo a última ocorrência com base em 'modifieddate', pois ela que indica a data da última modificação nos dados
#      Importante, pois se houver erro na ingestão (duplicação), mantém os dados íntegros.

Linhas após remover duplicatas com base em 'productid' e 'locationid': 1069


In [10]:
# Ordenar e exibir o df por 'productid'
production_productinventory = production_productinventory.sort_values(by=['productid'])

print(production_productinventory)

       productid  locationid shelf  bin  quantity                               rowguid              modifieddate
1368           1           6     B    5       324  d4544d7d-caf5-46b3-ab22-5718dcc26b5e 2014-08-08 00:00:00+00:00
720            1          50     A    5       353  bff7dc60-96a8-43ca-81a7-d6d2ed3000a8 2014-08-08 00:00:00+00:00
240            1           1     A    1       408  47a24246-6c43-48eb-968f-025738a8a410 2014-08-08 00:00:00+00:00
924            2           1     A    2       427  f407c07a-ca14-4684-a02c-608bd00c2233 2014-08-08 00:00:00+00:00
1332           2           6     B    1       318  ca1ff2f4-48fb-4960-8d92-3940b633e4c1 2014-08-08 00:00:00+00:00
...          ...         ...   ...  ...       ...                                   ...                       ...
10452        997           7   N/A    0       123  540b82be-09e5-4e99-9b7b-835ffab06950 2013-04-30 00:00:00+00:00
10392        998          60   N/A    0        56  5021e7ea-ce96-433c-860a-b9e57a45ef03 

In [11]:
# Iterar por todas as colunas do df, para verificar valores ausentes

# Verificar valores ausentes na coluna
for column in production_productinventory.columns:   
    missing_rows = production_productinventory[production_productinventory[column].isnull()]
    print(f"Coluna '{column}': {missing_rows.shape[0]} linhas ausentes.")
    
# Mostrar as primeiras linhas ausentes, se preciso for, limitar o head() para dar menos outputs ou limitar os outputs
    if not missing_rows.empty:
        print(f"Exibindo as primeiras linhas com valores ausentes em '{column}':")
        print(missing_rows.head(), "\n")
    else:
        print(f"Nenhuma linha com valores ausentes em '{column}'.\n")

Coluna 'productid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'productid'.

Coluna 'locationid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'locationid'.

Coluna 'shelf': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'shelf'.

Coluna 'bin': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'bin'.

Coluna 'quantity': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'quantity'.

Coluna 'rowguid': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'rowguid'.

Coluna 'modifieddate': 0 linhas ausentes.
Nenhuma linha com valores ausentes em 'modifieddate'.



In [12]:
# Valores únicos por coluna, para verificar se colunas como flags, normalmente booleanas, possuem apenas 1 ou 2 valores.

valores_unicos = production_productinventory.nunique(dropna=False)

print("Valores únicos incluindo NaN:")
print(valores_unicos)

Valores únicos incluindo NaN:
productid        432
locationid        14
shelf             21
bin               62
quantity         343
rowguid         1069
modifieddate      24
dtype: int64


In [13]:
#verificar informações do df
production_productinventory.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1069 entries, 1368 to 11736
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   productid     1069 non-null   Int64              
 1   locationid    1069 non-null   Int64              
 2   shelf         1069 non-null   object             
 3   bin           1069 non-null   Int64              
 4   quantity      1069 non-null   Int64              
 5   rowguid       1069 non-null   object             
 6   modifieddate  1069 non-null   datetime64[us, UTC]
dtypes: Int64(4), datetime64[us, UTC](1), object(2)
memory usage: 71.0+ KB


In [14]:
# Variáveis quantitativas*: estatísticas descritivas para verificar se ainda há o que ser feito antes de exportar os dados ao BigQuery

# Identificar colunas numéricas para análise de outliers
numeric_columns = ['quantity']

# Estatísticas Descritivas das colunas numéricas*
print(production_productinventory[numeric_columns].describe())

# Cálculo de limites para outliers (IQR)**
for col in numeric_columns:
    q1 = production_productinventory[col].quantile(0.25)
    q3 = production_productinventory[col].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    
    # Exibir os limites
    print(f"\nColuna: {col}")
    print(f"Limite inferior: {lower_bound}, Limite superior: {upper_bound}")
    
    # Detecção e Análise de Outliers***
    outliers = production_productinventory[(production_productinventory[col] < lower_bound) | (production_productinventory[col] > upper_bound)]
    print(f"Outliers detectados ({len(outliers)}):")
    print(outliers[[col]])


#doc*: variáveis quantitativas são um tipo de dado que pode ser representado por números e medidas objetivas
#doc*: realizar estatísticas descritivas para entender a centralidade e variação dos dados (valores médios, mínimos, máximos, etc.)
#doc**: calcular limites para identificar outliers (valores extremos que podem indicar erros ou casos excepcionais nos dados)
#doc***: verificar a existência de outliers para decidir ações como remoção, substituição ou tratamento, garantindo qualidade dos dados
#doc****: as colunas analisadas não apresentam outliers, pois os dados estão dentro dos limites esperados, sugerindo que não há necessidade de tratamento adicional para valores extremos. Isso indica boa qualidade dos dados para essas variáveis e que elas estão prontas para serem exportadas ou utilizadas em análises e modelos

       quantity
count   1069.00
mean     314.29
std      189.85
min        0.00
25%      140.00
50%      299.00
75%      481.00
max      924.00

Coluna: quantity
Limite inferior: -371.5, Limite superior: 992.5
Outliers detectados (0):
Empty DataFrame
Columns: [quantity]
Index: []


In [15]:
# Atualizar o dicionário df_processados com o df ajustado
df_processados['production_productinventory'] = production_productinventory

In [16]:
# Padronizar colunas com valores textuais
production_productinventory['shelf'] = production_productinventory['shelf'].str.strip().str.upper()
production_productinventory['rowguid'] = production_productinventory['rowguid'].str.strip().str.upper()

print(production_productinventory.head())

#doc: padronizar as strings nessa etapa, contribui para a execução das demais etapas do pipeline

      productid  locationid shelf  bin  quantity                               rowguid              modifieddate
1368          1           6     B    5       324  D4544D7D-CAF5-46B3-AB22-5718DCC26B5E 2014-08-08 00:00:00+00:00
720           1          50     A    5       353  BFF7DC60-96A8-43CA-81A7-D6D2ED3000A8 2014-08-08 00:00:00+00:00
240           1           1     A    1       408  47A24246-6C43-48EB-968F-025738A8A410 2014-08-08 00:00:00+00:00
924           2           1     A    2       427  F407C07A-CA14-4684-A02C-608BD00C2233 2014-08-08 00:00:00+00:00
1332          2           6     B    1       318  CA1FF2F4-48FB-4960-8D92-3940B633E4C1 2014-08-08 00:00:00+00:00


In [17]:
# Garantir que apenas tabelas únicas sejam exportadas
unique_df_processados = {k: v for k, v in df_processados.items()}

# Exportar tabelas para o BigQuery
for table_name, df_cleaned in unique_df_processados.items():
 
    output_table = f"{output_dataset}.{table_name}"
   
    job_config = bigquery.LoadJobConfig(
        write_disposition="WRITE_TRUNCATE"  
    )
    
    job = client.load_table_from_dataframe(df_cleaned, output_table, job_config=job_config)
    job.result()

    print(f"Tabela {table_name} exportada com sucesso para {output_table}.")

Tabela production_productinventory exportada com sucesso para desafioadventureworks-446600.raw_data_cleaned.production_productinventory.
